# DESAFIO CIENTISTA DE DADOS - INDICIUM

Participante: João Pedro Coelho Barbosa

### Nesse desafio vamos explorar de forma analítica um **Dataset** constando dados acerca da produção, faturamento e avaliação de filmes no IMDB com intuito de elaborar um algoritmo de predição de notas no IMDB
> Neste **Notebook** iremos abordar tópicos importantes acerca da modelagem do algoritmo de Machine Learning:
>>- Modelagem dos dados de treino e teste e Engenharia de Features _(Feature Engineering)_
>>- Validação dos modelos
>>- Otimização de hiper parâmetros 
>>- Escolha do modelo final para predição envolvendo o **Dataset `desafio_indicium_imdb.csv `**
---
### Principais desafios encontrados na Modelagem:
1.  Engenharia de Features:
- A modelagem dos dados para inserção nos modelos foi o principal desafio encontrado nessa jornada 

- Visto que, das 16 colunas do dataset original temos 5 numéricas, com as outras colunas constando o título do filme, onde não agregam de forma concisa nas features do modelo, além dessas também temos a classificação indicativa, os gêneros, o diretor e as estrelas que atuaram no filme.

- Com isso, a meta girou em torno de buscar formas de criar features para modelos constando a relevância dos diretores e atores presentes na base

- Técnicas como `OneHotEnconder` e `get_dummies` buscam expandir colunas categóricas de forma a agregar peso às features do modelo, transformando tais variáveis em diversas outras colunas booleanas de modo a apontar a categoria daquele dado de forma numérica, visto que os algoritmos "só entendem" números

- Todavia, o grande número de combinações de gêneros (mais de 170 combinações), de diretores (mais de 400) e atores (mais de 1900) presentes na base tornaria inviável aplicar tais técnicas, já que seria criado um grande número de features aos dados de treino e teste do modelo, fazendo com que ao invés do modelo generalizar e entender os dados, fortalecendo suas predições, o modelo iria somente decorar os dados causando `overfitting`

- Então criou 3 funções diferentes com foco em tratar esses dados:
    - `fe_genre`: Esta função cria colunas com variáveis boolenas (1 e 0 - Pertence ou não pertence ao gênero) para cada gênero presente nas combinações, reduzindo o total de 172 combinações de gênero para 21 gêneros e no caso, 21 colunas de gênero, onde ao invés de ter uma coluna que conste todos os gêneros de um filme, teremos colunas com 1 para os gêneros em que aquele filme pertence e colunas com 0 para os que aquele filme não pertecem, agregando informações de gêneros ao modelo

    - `fe_directors`: Como temos 402 diretores diferentes, temos que buscar alguma lógica que agregue informação do peso daquele diretor na qualidade do filme que ele dirige, assim apartir das colunas `Meta_score` e `No_of_votes` (Duas colunas com melhor correlação com a `IMDB_Rating`), criou-se TopNs, ou seja, buscou-se classificar cada diretor por quão bem ele é avaliado no Meta Score e quantidade de votos médio de seus filmes, visando entender também, sua popularidade e assim, ao invés de criar 400 colunas booleanas relacionando diretores aos seus filmes, criou-se colunas booleanas Top10, Top25 e Top50 indicando se aquele filme é dirigido por um diretor dentro daquele escopo TopN levantado, agregando a importância do diretor apartir do quão bem avaliado e popular frente à crítica ele é

    - `fe_actors`: Seguiu-se a mesma lógica por trás da `fe_director`, todavia como o dataset apresenta 4 colunas diferentes para atores constando as 4 estrelas principais do filme, onde, por exemplo, um mesmo ator pode aparecer com a principal estrela em um filme e segunda principal estrela de outro, torna-se necessário levantar individualmente cada ator concatenando essas colunas com um `.groupby()` utilizando as métricas `Meta_score` e `No_of_votes` e só em seguida levantar os Top10, Top25 e Top50 de cada uma

    > Uma outra abordagem interessante, mas que poderia induzir um **overfitting** seria avaliar da mesma forma, porém com o impacto de cada ator em cada posição de protagonismo num filme
    >> Por exemplo: Marlon Brando pode ter filmes super bem avaliados sendo o protagonista do filme, entretando como estrela secundária e terciária sua média de notas pode ser diferentes. Todavia, um nível de detalhismo muito grande e grande agregações de variável costuma tender a modelos com overfitting.

---

2. Escolha dos modelos
- Para este desafio utilizamos três modelos principais:
    - Um Baseline - Regressão Linear:
    >Modelo clássico de predição que busca encontrar uma reta que generalize a distribuição dos dados, utiliza um algoritmo iterativo que busca encontrar a menor soma das distâncias dos para a reta, quando tratamos de multidimensionalidade esse conceito se torna mais "abstrato" mas sua funcionalidade ainda é muito útil, tornando bastante usado como **_Baseline_** para estudo de predição.
            ![](https://imgur.com/t1admTB.png)
    - Um algoritmo de Bagging - Random Forest Regressor:
    >Bagging vem de *Bootstrap Aggregating*:
    >>- Bootstrap = criar várias amostras diferentes dos dados originais (com reposição).
    >>- Aggregating = juntar as previsões de todos os modelos (por média ou votação)
            >>>**Funcionamento:**
            >>>1. Temos o conjunto de dados inicial.
            >>>2. Gera diversas amostras aleatórias com reposição a partir desse conjunto de dados.
            >>>3. Para cada amostra aleatória gerada, treina um modelo diferente (uma árvore de decisão no nosso caso).
            >>>4. Ao final, junta os resultados de todos os modelos. Como é uma classificação, o resultado será a votação da maioria.   
            ![](https://i.imgur.com/EGDJjvk.png)
    - Um algoritmo de Boosting - XGBoost Regressor:
    >Boosting é uma técnica que vai aprendendo com erros dos preditores anteriores, ou seja, modelos simples que são treinados em sequência, onde cada um tenta corrigir os erros dos anteriores.:
    >>- Bootstrap = criar várias amostras diferentes dos dados originais (com reposição).
    >>- Aggregating = juntar as previsões de todos os modelos (por média ou votação)
            >>>**Funcionamento:**
            >>>1. Temos a base de dados inicial
            >>>2. Treinamos o primeiro modelo
            >>>3. Avaliamos onde ele errou
            >>>4. Dá mais peso para os exemplos onde ele errou
            >>>5. Treina o segundo modelo, porém focando em corrigir esses erros
            >>>6. Repete o processo, até o total de modelos indicados (no nosso caso o número de árvores de decisão)
            >>>7. Combina todas as previsões (Geralmente usando uma média ponderada ou soma com pesos) 
            ![](https://i.imgur.com/EGDJjvk.png)

- Outros algoritmos especializados em predições:
    - ARIMA _(AutoRegressive Integrated Moving Average)_ é um modelo clássico de séries temporais que captura tendências e padrões passados para prever valores futuros.
    - SARIMA _(Seasonal ARIMA)_ é uma extensão do ARIMA que também leva em conta a sazonalidade, sendo ideal para dados com ciclos regulares, como vendas mensais ou produção de energia através de fontes intermitentes como solar e eólica.
    - E algoritmos de _**Deep Learning**_ como LSTM _(Long Short-Term Memory)_ que busca "memorizar" valores antigos e sua influência em valores futuros

---

3. Métricas de Avaliação:
- RMSE _(Root Mean Squared Error)_ ou Raiz Quadrada do Erro Quadrático Médio
    - O RMSE mede a diferença média entre os valores previstos pelo modelo e os valores reais, penalizando erros maiores de forma mais intensa. Quanto menor o RMSE, melhor a performance do modelo.
    **Fórmula:**  
    $$
    \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
    $$
      
    **Onde:**
    - **$y_i$** = valor real da observação $i$
    - **$\hat{y}_i$** = valor previsto pelo modelo para a observação $i$
    - **$n$** = número total de observações

- R² _(Coeficiente de Determinação)_

    - O R² indica a proporção da variabilidade dos dados que é explicada pelo modelo. Varia entre 0 e 1, sendo que valores mais próximos de 1 indicam que o modelo consegue explicar bem a variação dos dados.  
    **Fórmula:**
    $$
    R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
    $$

    **Onde:**
    - **$y_i$** = valor real da observação $i$
    - **$\hat{y}_i$** = valor previsto pelo modelo para a observação $i$
    - **$\bar{y}$** = média dos valores reais
    - **$n$** = número total de observações

Em outras palavras:
- RMSE baixo → previsões próximas aos valores reais.  
- R² alto → modelo explica bem a variabilidade dos dados.

---

4. Otimização de Hiperparâmetros:
- A Otimização de hiperparâmetros é um passo crucial na decisão final da escolha do modelo.
- Para esse projeto, têm-se três modelos principais sendo estudados, o papel da otimização de hiperparâmetros é se utilizar de ténicas de validação, no caso deste estudo utilizou o `KFold`(Que se trata de uma téncnica de validação cruzada que foca em avaliar desempenho de diferentes modelos), e _pipelines_ de machine learning para avaliar qual a melhor configuração de hiperparâmetros para aqueles modelos.
- Ela funciona iterando os modelos sob diferentes espaços de buscas de hiperparâmetros pré-definidas e avaliando as métricas abordadas acima, com foco em entender qual configuração atende às melhores métricas para aquele modelo
    - O Algoritmo de otimização utilizado foi o `optuna`, onde precisamos definir a função `objective` que seria o "objetivo de estudo" do algoritmo de otimização
    - É necessário definir o que também será a referência para a otimização, ou seja, o que será de fato otimizado, isto é, a métrica utilizada como base para ser otimizada e o sentido de sua otimização, por exemplo:
    > Se utilizarmos o RMSE temos que buscar o sentido de minimização da métrica, tornando a menor e aumentando a proximidade dos valores previstos com os reais, se usarmos o R² temos que buscar o sentido de maximização da métrica, tornando a maior e aumentando a capacidade do modelo de generalizar a variabilidade dos dados
    - Para esse caso, a melhor métrica e sentido de otimização é: Minimização do RMSE buscando ter modelos que estimem com o máximo de proximidade as notas do IMDB

### Principais resultados

### Tabela de Resultados dos Modelos

| Modelo | R2 | RMSE |
|:---|:---|:---|
| LinearRegression | 0.5850 | 0.1864 |
| RandomForest | 0.5947 | 0.1842 |
| XGBoost | 0.5110 | 0.2024 |

- RandomForest apresenta as melhores métricas e foi o modelo escolhido para o deploy em .pkl e predição da nota IMDB do filme Shawshank Redemption
> Interessante abordar a versatilidade da clássica regressão linear, se destacando a frente do XGBoost e bem próximo ao RandomForest em questão de métricas
---

### Melhores Parâmetros dos Modelos

| Modelo | Parâmetros |
|:---|:---|
| LinearRegression | `{'fit_intercept': True}` |
| RandomForest | `{'n_estimators': 100, 'max_depth': 19, 'min_samples_leaf': 1}` |
| XGBoost | `{'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.14}` |

### Nota IMDB estimada para Shawshank Redemption e conclusões tiradas do estudo

- O filme Shawshank Redemptio apresenta uma nota IMDB de 9.3, sendo a maior nota da história do site para um filme, se tratando claramente de um outlier que estava fora do escopo da base de treino e teste, fator que, de certo modo, não só dificulta a acertividade do modelo como praticamente impossibilita o modelo de prever tal valor, visto que, o modelo não "faz ideia" da possibilidade de uma nota desse valor já que ele aprendeu só até 9.2 que era o segundo maior valor de nota histórica dada pelo IMDB ao filme The Godfather
- Todavia, o modelo previu uma nota de 8.8 para o filme, se tratando de uma nota ainda muito boa e bem destacada frente aos demais filme da base, pertencendo a um grupo seleto de notas que se encaixariam no top10 histórico de maiores notas ja dadas pelo IMDB. Então, de certo modo o modelo previu que o filme estaria dentro do top10 mais bem avaliados filmes na história do IMDB, um fator que reflete em si, também, na forma de tratamento de dados e criação de features realizada nesse estudo.
> Um conclusão e ideia interessante seria tentar abordar topNs mais restritos para diretores e atores, como top5, top10 e por ai vai

__Este foi um ótimo estudo onde pude abordar técnicas previamente estudadas acerca dos tópicos que envolvem a Ciência de Dados no geral, tive que ir atrás de alguns conhecimentos que não tinha de forma concreta para embasar o estudo e as conclusões que tirei acerca do mesmo. Empolgante a parte de explorar formas de criar novas features com foco em trazer uma capacidade de generalização maior ao modelo através dos dados.__

## 1. Importação de bibliotecas e utilitários

Este trecho de código importa bibliotecas, configurações e variáveis necessárias para a análise exploratória e processamento de dados.

| Biblioteca | Função principal |
|------------|-----------------|
| **pandas** | Manipulação de dados em DataFrames |
| **numpy** | Operações matemáticas e vetoriais |
| **matplotlib.pyplot** | Visualizações gráficas básicas |
| **seaborn** | Visualizações estatísticas mais elaboradas |
| **sklearn.model_selection** | Divisão de dados, validação cruzada e busca de hiperparâmetros |
| **sklearn.preprocessing** | Pré-processamento de dados (ex.: escalonamento, codificação) |
| **sklearn.compose** | Transformações de colunas específicas via `ColumnTransformer` |
| **sklearn.pipeline** | Criação de pipelines para modelagem e pré-processamento |
| **sklearn.ensemble** | Modelos de ensemble, como Random Forest |
| **sklearn.metrics** | Avaliação de modelos (ex.: RMSE, R²) |
| **sklearn.linear_model** | Modelos lineares, como regressão linear |
| **xgboost** | Modelos de boosting, como XGBRegressor |
| **optuna** | Otimização de hiperparâmetros de forma automatizada |
| **joblib** | Salvamento e carregamento de modelos treinados |
| **functools.partial** | Criação de funções parciais para parametrização flexível |
| **sys** | Configuração do path da pasta local e acesso a funcionalidades do sistema |
| **src.eda_utils** | Arquivo de utilidades utilizado na EDA e na modelagem |



In [129]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import joblib
from functools import partial

import sys
sys.path.append("..") 
from src.eda_utils import conv_numerico


## 2. Definindo o path e realizando a ingestão da base no script

- Definindo o path do arquivo na pasta **/processed** buscando seguir as boas práticas de organização de diretórios
- Leitura do **.csv** e conversão para um dataframe utilizando pandas


In [130]:
path = '../data/processed/data_cleaned.csv'
df = pd.read_csv(path)
df.head(3)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
1,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
2,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0


## 3. Definindo colunas categóricas, numéricas e para dropar do dataset
- As colunas categóricas e numéricas tem foco em criar uma melhor organização e boas práticas para o momento do pré-processamento de dados
- As colunas para dropar tem foco em limpar o dataset, preparando os dados para treino e teste

In [131]:
# Colunas numéricas e categóricas para pré-processamento e drop de colunas nos dados de treino e teste

num_cols = ['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes','Gross']
cat_cols = ['Certificate']
drop_cols = ['Series_Title', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'IMDB_Rating']


## 4. Função de _Feature Engineering_ para a coluna `Genre`
- A função expande inicialmente a coluna de gênero em 3 diferentes colunas e seguida concatena elas em `generos`
- Como alguns filmes apresentam só 1 ou 2 gêneros, exisitirão colunas com dados nulos
- Logo mais é feito o drop desses dados nulos em `generos` e um `.strip()` seguido de um `.unique()` com a intenção de limpar os espaços vazios e buscar os valores únicos, ou seja, levantar todos os temas presentes na base e salvando em `generos_unicos`
- Cria-se um loop para iterar `genero` em `generos_unicos` com a ideia de criar uma coluna para cada gênero
- Assim foi-se criado novas 21 colunas

In [132]:
def fe_genre (df):
    df[['Genre1', 'Genre2', 'Genre3']] = df['Genre'].str.split(',', expand=True)

    generos = pd.concat([df['Genre1'], 
                        df['Genre2'], 
                        df['Genre3']])

    # Dropando os valores NaN, removendo espaços em branco e pegando os valores únicos
    generos_unicos = generos.dropna().str.strip().unique()
    print(f"Número de gêneros únicos na base: {len(generos_unicos)}\n")
    print(f"Gêneros presentes na base:{generos_unicos}\n")

    # Criando colunas para cada gênero com variável booleana
    for genero in generos_unicos:
        df[genero] = df['Genre'].str.contains(genero, na=False).astype(int)

    # Dropando as colunas 'Genre', 'Genre1', 'Genre2' e 'Genre3'
    df_genres = df.drop(columns=['Genre', 'Genre1', 'Genre2', 'Genre3'])

    # Shape do novo dataframe
    print(f"Shape do dataframe: {df_genres.shape}\n")
    print(f"Colunas do novo dataframe: {df_genres.columns}\n")

    return df_genres

# Visualizando o novo dataframe
df_genre_vis = df.copy()
visualizar_df_genre = fe_genre(df_genre_vis)
visualizar_df_genre.iloc[:,:21].head()

Número de gêneros únicos na base: 21

Gêneros presentes na base:['Crime' 'Action' 'Biography' 'Drama' 'Western' 'Comedy' 'Adventure'
 'Animation' 'Horror' 'Mystery' 'Film-Noir' 'Family' 'Romance' 'Sci-Fi'
 'War' 'Music' 'Thriller' 'Musical' 'Fantasy' 'Sport' 'History']

Shape do dataframe: (712, 35)

Colunas do novo dataframe: Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime',
       'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2',
       'Star3', 'Star4', 'No_of_Votes', 'Gross', 'Crime', 'Action',
       'Biography', 'Drama', 'Western', 'Comedy', 'Adventure', 'Animation',
       'Horror', 'Mystery', 'Film-Noir', 'Family', 'Romance', 'Sci-Fi', 'War',
       'Music', 'Thriller', 'Musical', 'Fantasy', 'Sport', 'History'],
      dtype='object')



,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,...,Star4,No_of_Votes,Gross,Crime,Action,Biography,Drama,Western,Comedy,Adventure
0,The Godfather,1972,A,175,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,...,Diane Keaton,1620367,134966411.0,1,0,0,1,0,0,0
1,The Dark Knight,2008,UA,152,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,...,Michael Caine,2303232,534858444.0,1,1,0,1,0,0,0
2,The Godfather: Part II,1974,A,202,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,...,Diane Keaton,1129952,57300000.0,1,0,0,1,0,0,0
3,12 Angry Men,1957,U,96,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,...,John Fiedler,689845,4360000.0,1,0,0,1,0,0,0
4,The Lord of the Rings: The Return of the King,2003,U,201,8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,...,Orlando Bloom,1642758,377845905.0,0,1,0,1,0,0,1


## 5. Função de _Feature Engineering_ para a coluna `Director`

- Cria-se 2 listas
    - Top50 diretores em média no Meta Score
    - Top50 diretores em média de número de votos
- Em seguida define-se uma função `criar_coluna_top_n` com foco em iterar sob as listas de top50s e extrair delas os top10, top25 e top50 melhores diretores para cada métrica
- E assim itera cada diretor sob a ideia de verificar se ele esta ou não em cada uma dessas listas, se tiver preenche com 1 se não tiver, preenche com 0
- Importante pontuar que, como são colunas geradas apartir de métricas extraídas de outras colunas, tais alterações devem ser feitas apos o `train_test_split()`, evitando _Data Leakage_ para os dados de teste
> _Data Leakage_ é quando "vazamos" informações dos dados de treino para os dados de teste, fazendo com que o modelo "pesque" no dados de treino informaçoes que ele utilizará nos dados de teste

In [133]:
def fe_directors(train_df, test_df):
    top_n = [10, 25, 50]

    # Criando listas com os 50 melhores diretores do treino
    top50_dir_metascore = train_df.groupby('Director')['Meta_score'].mean().sort_values(ascending=False).head(50).index.tolist()
    top50_dir_no_of_votes = train_df.groupby('Director')['No_of_Votes'].mean().sort_values(ascending=False).head(50).index.tolist()
    #top50_dir_filmes = train_df['Director'].value_counts().head(50).index.tolist()

    # Função para criar colunas booleanas no dataset
    def criar_colunas_top_n(df, top_lists, top_n):

        for n in top_n:
            df[f'top{n}_dir_metascore'] = df['Director'].isin(top_lists['metascore'][:n]).astype(int)
            df[f'top{n}_dir_no_of_votes'] = df['Director'].isin(top_lists['no_of_votes'][:n]).astype(int)
            #df[f'top{n}_dir_filmes'] = df['Director'].isin(top_lists['filmes'][:n]).astype(int)

    # Dicionário com listas top50
    top_lists = {
        'metascore': top50_dir_metascore,
        'no_of_votes': top50_dir_no_of_votes,
        #'filmes': top50_dir_filmes
    }

    # Criando as colunas no dataset de treino
    criar_colunas_top_n(train_df, top_lists, top_n)

    # Criando as colunas no dataset de teste
    criar_colunas_top_n(test_df, top_lists, top_n)

    return train_df, test_df

## 6. Função de _Feature Engineering_ para as colunas `StarN`

- Inicialmente, o código itera sobre cada uma das colunas de atores (`Star1`, `Star2`, `Star3`, `Star4`).
- Para cada coluna, um DataFrame temporário é criado, contendo o nome do ator e as colunas de `Meta_score`, `No_of_Votes` e incluindo um novo TopN referente a quantidade de filmes realizados, com a ideia de amenizar o impacto de atores que realizaram poucos filmes porém bem avaliados e vice-versa.
- Em seguida, essas colunas são renomeadas para padronizar o nome do ator para `Ator`.
- Por fim, todos os DataFrames temporários são concatenados em um único DataFrame chamado `atores_df`, e as linhas com valores ausentes são removidas.
- Logo mais, calcula-se as métricas de forma semelhante à função `fe_directors` criando colunas para os TopNs atores em cada métrica iterando sob cada ator e preenchendo cada coluna com a quantidade de atores referentes a cada TopN, por exemplo:
> Para a métrica Meta Score vamos pegar o filme The Godfather com Marlon Brandon e Al Pacino como estrelas, onde ambos os atores são top10 Meta Score, logo a coluna ficaria com o 2, indicando que tal filme tem 2 atores top10 dentre as 4 estrelas presentes no filme

In [142]:
def fe_actors(train_df, test_df):
    # Feature Engineering da coluna 'Actors'

    atores = []

    for col in ['Star1', 'Star2', 'Star3', 'Star4']:
        temp_df = train_df[[col, 'Meta_score', 'No_of_Votes']].copy()
        temp_df.columns = ['Ator', 'Meta_score', 'No_of_Votes']
        atores.append(temp_df)

    atores_df = pd.concat(atores).dropna()

    # Calcular médias e quantidade de filmes por ator
    ator_metascore = atores_df.groupby('Ator')['Meta_score'].mean()
    ator_votes = atores_df.groupby('Ator')['No_of_Votes'].mean()
    ator_filmes = atores_df.groupby('Ator').size()

    # Top atores usando loop
    top_n = [10, 25, 50]

    # Dicionários para guardar os tops
    top_metascore = {}
    top_votes = {}
    top_filmcount = {}

    for n in top_n:
        top_metascore[n] = set(ator_metascore.nlargest(n).index)
        top_votes[n] = set(ator_votes.nlargest(n).index)
        top_filmcount[n] = set(ator_filmes.nlargest(n).index)

    # Função para contar
    def top_atores(row, top_set):
        atores = [row['Star1'], row['Star2'], row['Star3'], row['Star4']]
        return sum(1 for ator in atores if ator in top_set)

    for n in top_n:
        train_df[f'top{n}_atores_metascore'] = train_df.apply(lambda x: top_atores(x, top_metascore[n]), axis=1)
        train_df[f'top{n}_atores_votes'] = train_df.apply(lambda x: top_atores(x, top_votes[n]), axis=1)
        train_df[f'top{n}_atores_filmes'] = train_df.apply(lambda x: top_atores(x, top_filmcount[n]), axis=1)

    for n in top_n:
        test_df[f'top{n}_atores_metascore'] = test_df.apply(lambda x: top_atores(x, top_metascore[n]), axis=1)
        test_df[f'top{n}_atores_votes'] = test_df.apply(lambda x: top_atores(x, top_votes[n]), axis=1)
        test_df[f'top{n}_atores_filmes'] = test_df.apply(lambda x: top_atores(x, top_filmcount[n]), axis=1)

    return train_df, test_df

## 7. Função para salvamento e preparo das bases de treino e teste
- Salva a base de treino processada em `../data/train/data_train_model.csv`
- Salva a base de teste processada em `../data/test/data_test_model.csv`


In [135]:
def preparar_e_salvar_dados(train_df, test_df, drop_cols):
    # Guardando títulos para referência
    train_titles = train_df['Series_Title'].reset_index(drop=True)
    test_titles = test_df['Series_Title'].reset_index(drop=True)

    # Definindo as features e o target
    X_train = train_df.drop(drop_cols, axis=1).reset_index(drop=True)  
    y_train = train_df['IMDB_Rating'].reset_index(drop=True)                 

    X_test = test_df.drop(drop_cols, axis=1).reset_index(drop=True)    
    y_test = test_df['IMDB_Rating'].reset_index(drop=True)                  

    # Verificando o shape dos datasets
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")

    # Verificando se há valores nulos nas features
    print(f"\nValores nulos em X_train: {X_train.isnull().sum().sum()}")
    print(f"Valores nulos em X_test: {X_test.isnull().sum().sum()}")

    # Verificando se há valores nulos na target
    print(f"\nValores nulos em y_train: {y_train.isnull().sum()}")
    print(f"Valores nulos em y_test: {y_test.isnull().sum()}")

    # Verificando as colunas
    print(f"Quantidade de colunas em X_train: {X_train.shape[1]}")
    print(f"Colunas em X_train: {X_train.columns.tolist()}")

    df_train_model = X_train.copy()
    df_train_model['IMDB_Rating'] = y_train
    df_train_model.to_csv('../data/train/data_train_model.csv', index=False)

    df_test_model = X_test.copy()
    df_test_model['IMDB_Rating'] = y_test
    df_test_model.to_csv('../data/test/data_test_model.csv', index=False)

    return train_titles, test_titles, X_train, y_train, X_test, y_test

## 8. Pipeline de processamento de dados
- Pipeline que centraliza as funções de criação de features e salvamento dos dados
- Ordem das suas tarefas segue como:
> `fe_genre()` -> `train_test_split()` -> `fe_directors()` -> `fe_actors()` -> `preparar_e_salvar_dados()`

In [136]:
# Pipeline de Feature Engineering

def pipeline_feature_engineering(df, drop_cols):
    # Cadeia de transformações
    df_processed = fe_genre(df.copy())
    
    train_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
    
    # Aplicando transformações nos datasets divididos
    train_df, test_df = fe_directors(train_df.copy(), test_df.copy())
    train_df, test_df = fe_actors(train_df, test_df)
    
    train_df.head()

    # Preparação final
    results = preparar_e_salvar_dados(train_df, test_df, drop_cols)
    
    return results

## 9. Definindo os espaços de busca do algoritmo de otimização de hiperparâmetros
- Definindo os parâmetros a serem otimizados e o espaço de busca de cada um, ou seja, quais valores de cada parâmetros serão estudados com foco em encontrar os melhores parâmetros para cada modelo

In [137]:
# Definindo espaço de busca para otimização de hiperparâmetros

def espaco_busca_lr(trial):
    # Espaço de busca para regressão linear
    return {
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
    }

def espaco_busca_rf(trial):
    # Espaço de busca para o random forest
    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'random_state': 42 
    }

def espaco_busca_xgb(trial):
    # Espaço de busca para o XGBoost
    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 400, step=50),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step=0.01),
        'random_state': 42,
        'eval_metric': 'rmse'
    }


## 10. Dicionário de configuração dos modelos
- Dicionário que consta cada modelo e seu espaço de busca
- Visa seguir boas práticas e tornar o código mais limpo e de fácil entendimento

In [138]:
# Modelos utilizados e seus respectivos espaços de busca

MODEL_CONFIG = {
    "LinearRegression": {
        "classe_modelo": LinearRegression,
        "espaco_busca_modelo": espaco_busca_lr
    },
    "RandomForest": {
        "classe_modelo": RandomForestRegressor,
        "espaco_busca_modelo": espaco_busca_rf
    },
    "XGBoost": {
        "classe_modelo": XGBRegressor,
        "espaco_busca_modelo": espaco_busca_xgb
    }
}

## 11. Função `objetive`
- Define o objeto de estudo do algoritmo de otimização de hiperparâmetros
- Validação cruzada utilizando `KFold`
- Criação da pipeline de pré-processamento utilizando algoritmos de transformação de variáveis:
    - `StandarScaler()` para variáveis numéricas, visando padronizar os dados numéricos:
            $$z = \frac{x - \mu}{\sigma}$$

        **Onde:**

        * **$x$**: é o valor original do dado.
        * **$\mu$**: é a média dos dados.
        * **$\sigma$**: é o desvio padrão dos dados.
        * **$z$**: é o valor padronizado.
    
    - `OneHotEncoder` para variáveis categóricas, buscando criar colunas booleanas que agreguem informação ao modelo acerca das diferentes categorias presentes nas colunas categóricas

- Pipeline de pré-processamento + algoritmo de machine learning
- `cross_val_score` com foco em extrair as metricas de cada modelo no processo de validação
- Cálculo do RMSE - métrica a ser otimizada pelo `optuna`

In [139]:
def objective(trial, classe_modelo, espaco_busca, X_train, y_train, cat_cols, num_cols):
    
    # Definindo o KFold para validação cruzada dos modelos
    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    # Configurando o espaço de busca e o modelo
    params = espaco_busca(trial)
    model = classe_modelo(**params)

    preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', num_cols),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
            ]
        )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Avaliando o modelo com validação cruzada
    scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

    # Valor de referência a ser otimizado
    mse_medio = -scores.mean()
    
    return mse_medio

12. Função de treinamento e otimização de hiperparâmetros
- A função `treinar_e_otimizar_modelos` é responsável por automatizar o fluxo de trabalho de treinamento de modelos.
- Inicialmente, ela executa o pipeline de **Feature Engineering** (`pipeline_feature_engineering`) para preparar os dados, dividindo-os em conjuntos de treino e teste.
- Em seguida, a função itera sobre cada modelo definido em `MODEL_CONFIG`.
- Para cada modelo, um estudo **Optuna** é criado para otimização de hiperparâmetros, onde a função `objective` é chamada para encontrar a melhor combinação de parâmetros, minimizando a métrica de erro (RMSE).
- Com os melhores parâmetros encontrados, a função treina uma **pipeline final**, que inclui o pré-processamento (`ColumnTransformer`) e o modelo (`regressor`), usando todo o conjunto de dados de treino.
- Por fim, o modelo treinado é avaliado no conjunto de teste, e suas métricas (`R2` e `RMSE`) são calculadas e exibidas para comparar o desempenho de cada modelo.
- A função retorna os modelos treinados, os melhores parâmetros e os resultados da avaliação no conjunto de teste.

In [140]:
# Otimização de hiperparâmetros utilizando a função otimizacao e objective no optuna, treinamento e avaliação dos modelos

def treinar_e_otimizar_modelos(MODEL_CONFIG, df_original, drop_cols, num_cols, cat_cols, n_trials=15):

    modelos_treinados = {}
    best_params = {}
    resultados_test = {}
    
    # Executando o pipeline de FE
    train_titles, test_titles, X_train, y_train, X_test, y_test = pipeline_feature_engineering(
        df_original, drop_cols
    )
    print(f"FE concluído. Shape treino: {X_train.shape}, Shape teste: {X_test.shape}")

    for nome_modelo, config in MODEL_CONFIG.items():
        # Criando estudo Optuna
        study = optuna.create_study(direction='minimize')

        # Função objective com as configurações do modelo e dados de treinamento para validação cruzada
        objective_dados = partial(
            objective,
            classe_modelo=config['classe_modelo'],
            espaco_busca=config['espaco_busca_modelo'],
            X_train=X_train,
            y_train=y_train,
            cat_cols=cat_cols,
            num_cols=num_cols
        )
        
        # Otimização de hiperparÇametros
        study.optimize(objective_dados, n_trials=n_trials)
        best_params[nome_modelo] = study.best_params
        print(f"Melhores parâmetros {nome_modelo}: {study.best_params}")

        # Treinando modelo final com melhores parâmetros
        modelo_final = config['classe_modelo'](**study.best_params)
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', num_cols),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
            ]
        )

        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', modelo_final)
        ])

        pipeline.fit(X_train, y_train)
        modelos_treinados[nome_modelo] = pipeline

        # Avaliando no conjunto de teste
        y_pred = pipeline.predict(X_test)
        resultados_test[nome_modelo] = {
            'R2': r2_score(y_test, y_pred),
            'RMSE': mean_squared_error(y_test, y_pred, squared=False)
        }

        print(f"Métricas no teste {nome_modelo}: {resultados_test[nome_modelo]}")

    # Comparando os resultados dos modelos
    print("\nResultados dos modelos no conjunto de teste:")

    for nome_modelo, metrics in resultados_test.items():
        print(f"{nome_modelo}: R2 = {metrics['R2']:.4f}, RMSE = {metrics['RMSE']:.4f}")
        print(f"Melhores parâmetros: {best_params[nome_modelo]}")

    print("\nTreinamento e otimização concluídos.")
    
    dados_processados = {
        'train_titles': train_titles,
        'test_titles': test_titles,
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

    return modelos_treinados, best_params, resultados_test, dados_processados

## 14. Avaliação e Comparação dos Modelos

- Esta etapa é crucial para entender qual modelo teve o melhor desempenho
- Os resultados obtidos no conjunto de teste são comparados para identificar o modelo mais robusto e com maior capacidade de generalização

    * **R2 (Coeficiente de Determinação):** Mede a proporção da variância na variável dependente que é previsível a partir das variáveis independentes. Quanto mais próximo de 1, melhor o modelo se ajusta aos dados.
    * **RMSE (Root Mean Square Error):** Avalia a diferença média entre os valores previstos e os valores reais. É uma medida de erro, onde valores mais baixos indicam melhor desempenho.

- É possível visualizar, também, a melhor configuração de hiperparâmetro para cada modelo

In [143]:
modelos, params, resultados, dados = treinar_e_otimizar_modelos(
    MODEL_CONFIG, 
    df,    
    drop_cols,        
    num_cols,       
    cat_cols,       
    n_trials=15)

[I 2025-09-06 16:13:49,271] A new study created in memory with name: no-name-d8732362-4bba-4837-9a83-20ffd93c421d
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:49,322] Trial 0 finished with value: 0.04846320939999391 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 0.04846320939999391.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:49,347] Trial 1 finished with value: 0.03792708478189042 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 0.03792708478189042.


Número de gêneros únicos na base: 21

Gêneros presentes na base:['Crime' 'Action' 'Biography' 'Drama' 'Western' 'Comedy' 'Adventure'
 'Animation' 'Horror' 'Mystery' 'Film-Noir' 'Family' 'Romance' 'Sci-Fi'
 'War' 'Music' 'Thriller' 'Musical' 'Fantasy' 'Sport' 'History']

Shape do dataframe: (712, 35)

Colunas do novo dataframe: Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime',
       'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2',
       'Star3', 'Star4', 'No_of_Votes', 'Gross', 'Crime', 'Action',
       'Biography', 'Drama', 'Western', 'Comedy', 'Adventure', 'Animation',
       'Horror', 'Mystery', 'Film-Noir', 'Family', 'Romance', 'Sci-Fi', 'War',
       'Music', 'Thriller', 'Musical', 'Fantasy', 'Sport', 'History'],
      dtype='object')

X_train shape: (569, 42)
y_train shape: (569,)
X_test shape: (143, 42)
y_test shape: (143,)

Valores nulos em X_train: 0
Valores nulos em X_test: 0

Valores nulos em y_train: 0
Valores nulos em y_test: 0
Quanti

c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:49,375] Trial 2 finished with value: 0.04846320939999391 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.03792708478189042.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:49,405] Trial 3 finished with value: 0.03792708478189042 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 0.03792708478189042.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warni

Melhores parâmetros LinearRegression: {'fit_intercept': True}
Métricas no teste LinearRegression: {'R2': 0.5850022623896449, 'RMSE': 0.18644698600676912}


c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:53,469] Trial 0 finished with value: 0.03235907688078857 and parameters: {'n_estimators': 700, 'max_depth': 29, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.03235907688078857.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:13:56,714] Trial 1 finished with value: 0.03673974846610234 and parameters: {'n_estimators': 900, 'max_depth': 20, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.03235907688078857.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] duri

...

c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-09-06 16:14:26,213] A new study created in memory with name: no-name-00e717a6-6ad6-4848-9e0d-867b83fcff55
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Métricas no teste RandomForest: {'R2': 0.6072058505061297, 'RMSE': 0.1813906962912218}


[I 2025-09-06 16:14:26,657] Trial 0 finished with value: 0.038703725148186895 and parameters: {'n_estimators': 350, 'max_depth': 17, 'learning_rate': 0.3}. Best is trial 0 with value: 0.038703725148186895.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:14:27,282] Trial 1 finished with value: 0.0409994757101935 and parameters: {'n_estimators': 100, 'max_depth': 29, 'learning_rate': 0.13}. Best is trial 0 with value: 0.038703725148186895.
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
[I 2025-09-06 16:14:27,988] Trial 2 finished with value: 0.04109336275992715 and parameters: {'n_estimators': 250, 'max_depth': 20, 'learni

Melhores parâmetros XGBoost: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.060000000000000005}
Métricas no teste XGBoost: {'R2': 0.5446614571006329, 'RMSE': 0.19529886051069925}

Resultados dos modelos no conjunto de teste:
LinearRegression: R2 = 0.5850, RMSE = 0.1864
Melhores parâmetros: {'fit_intercept': True}
RandomForest: R2 = 0.6072, RMSE = 0.1814
Melhores parâmetros: {'n_estimators': 700, 'max_depth': 29, 'min_samples_leaf': 1}
XGBoost: R2 = 0.5447, RMSE = 0.1953
Melhores parâmetros: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.060000000000000005}

Treinamento e otimização concluídos.


c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## 15. Selecionando o melhor modelo e extraindo um .csv com as previsões para cada filme na base de teste
- Seleção do Modelo: O modelo **Random Forest** é escolhido como o "melhor modelo" com base em sua performance superior, identificada na etapa de avaliação anterior.
- Previsões: O modelo selecionado faz previsões (`y_pred`) no conjunto de dados de teste.
- Comparação e Exportação: As previsões são organizadas em um DataFrame junto com o valor real do `IMDB_Real` e o nome do filme. A coluna de previsão (`IMDB_Previsto`) é arredondada para uma casa decimal para maior legibilidade. Por fim, o DataFrame é exportado para um arquivo CSV (`imdb_rating_pred.csv`) na pasta `../data/predicted/`

In [144]:
best_model = modelos['RandomForest']
def comparar_imdb_previsto(best_model, dados_processados, nomes_filmes_col='test_titles'):
    # Dados de teste
    X_test = dados_processados['X_test']
    y_test = dados_processados['y_test']
    test_titles = dados_processados[nomes_filmes_col]

    # Previsões
    y_pred = best_model.predict(X_test)
    
    # Cria o DataFrame
    df_plot = pd.DataFrame({
        'Filme': test_titles,
        'IMDB_Real': y_test,
        'IMDB_Previsto': y_pred
    })
    
    # Arredonda a coluna 'IMDB_Previsto' para 1 casa decimal
    df_plot['IMDB_Previsto'] = df_plot['IMDB_Previsto'].round(1)

    # Exportando para csv
    df_plot.to_csv("../data/predicted/imdb_rating_pred.csv", sep=";")
    
    # Exibe o DataFrame
    print(df_plot)

# Chame a função para criar e mostrar o DataFrame com os valores arredondados
comparar_imdb_previsto(best_model, dados, nomes_filmes_col='test_titles')

                       Filme  IMDB_Real  IMDB_Previsto
0    Star Trek Into Darkness        7.7            7.8
1               Kaze tachinu        7.8            7.9
2                  Gully Boy        8.0            8.0
3            The Incredibles        8.0            7.9
4                  Cast Away        7.8            7.8
..                       ...        ...            ...
138           Doctor Zhivago        8.0            7.9
139       Back to the Future        8.5            8.4
140      There Will Be Blood        8.2            7.9
141     (500) Days of Summer        7.7            7.8
142                   WALL·E        8.4            8.1

[143 rows x 3 columns]


c:\Users\joaop\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
# Variaveis do filme Shawshank Redemption

filme = {'Series_Title': 'The Shawshank Redemption',
        'Released_Year': '1994',
        'Certificate': 'A',
        'Runtime': '142 min',
        'Genre': 'Drama',
        'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
        'Meta_score': 80.0,
        'Director': 'Frank Darabont',
        'Star1': 'Tim Robbins',
        'Star2': 'Morgan Freeman',
        'Star3': 'Bob Gunton',
        'Star4': 'William Sadler',
        'No_of_Votes': 2343110,
        'Gross': '28,341,469'}

In [ ]:
# Exportando melhor modelo para .pkl
joblib.dump(best_model, "..\models\RandomForest.pkl")

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\joaop\AppData\Local\Temp\ipykernel_4392\1955653615.py:2: SyntaxWarning: invalid escape sequence '\m'
  joblib.dump(best_model, "..\models\RandomForest.pkl")


['..\\models\\RandomForest.pkl']

## 16. Função de predição de `IMDB_Rating` para novos filmes

- **Carregamento do Modelo:** O modelo **Random Forest** previamente treinado é carregado do arquivo `RandomForest.pkl` utilizando a biblioteca `joblib`.
- **Preparação dos Dados:** Uma função, `prever_imdb_rating`, é criada para receber os dados de um filme (como um dicionário) e prepará-los para a previsão. Isso inclui a conversão dos dados para um formato de DataFrame e a aplicação das transformações numéricas necessárias com a função `conv_numerico`.
- **Execução da Previsão:** O modelo carregado (`melhor_modelo`) é usado para prever o `IMDB_Rating` do filme preparado.

In [ ]:
# Prevendo IMDB_Rating de filmes

melhor_modelo = joblib.load("../models/RandomForest.pkl")

# Prevendo IMDB_Rating de filmes

melhor_modelo = joblib.load("../models/RandomForest.pkl")

def prever_imdb_rating(filme, melhor_modelo):
    melhor_modelo = modelos['RandomForest']
    
    df_filme = pd.DataFrame([filme])
    df_filme = conv_numerico(df_filme)
    imdb_rating_pred = melhor_modelo.predict(df_filme)
    
    # A linha abaixo foi corrigida para pegar apenas o valor do título
    titulo_filme = df_filme['Series_Title'].iloc[0]
    
    print(f"IMDB previsto para o filme {titulo_filme} é: {imdb_rating_pred[0]:.1f}\nO real valor da nota IMDB do filme é 9.3")
    

    return df_filme

prever_imdb_rating(filme,melhor_modelo)

IMDB previsto para o filme The Shawshank Redemption é: 8.8
O real valor da nota IMDB do filme é 9.3


,Series_Title,Released_Year,Certificate,Runtime,Genre,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Shawshank Redemption,1994,A,142,Drama,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469.0
